In [7]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

from QuantumCircuit import QuantumCircuit

In [16]:
qc = QuantumCircuit(1)
qc.h(0).execute()
qc.bloch()

AttributeError: 'NoneType' object has no attribute 'execute'